In [ ]:
import sys, time, os, asyncio
import scipy.stats as stats
from scipy.signal import find_peaks
from scipy import signal
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time
from matplotlib.ticker import FormatStrFormatter
from lsst_efd_client import EfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient, clipDataToEvent, calcNextDay
from datetime import datetime
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
def get_freq_psd(vals, timestep):

    """
    Calculates the frequency power spectrum of a signal.

    Args:
        vals (np.array): The signal values.
        timestep (float): The time step between samples.

    Returns:
        tuple: The frequencies and power spectral density.
    """

    # Remove the mean from the signal.

    meanval = np.mean(vals)
    signal = vals - meanval

    # Calculate the length of the signal.

    N = len(signal)

    # Calculate the power spectral density.

    psd = np.abs(np.fft.rfft(np.array(signal) * 1)) ** 2

    # Calculate the frequencies.

    frequencies = np.fft.rfftfreq(N, timestep)
    return (frequencies, psd)


def add_timestamp(data):
    """
    Adds a correct timestamp column in UTC format to the provided data if
    not present.

    Parameters
    ----------
    data : DataFrame
        The data to which the timestamp will be added.

    Returns
    -------
    DataFrame
        The data with the added 'snd_timestamp_utc' column.
    """
    if "snd_timestamp_utc" not in data.columns:
        data["snd_timestamp_utc"] = Time(
            data["private_sndStamp"], format="unix_tai"
        ).unix
    return data


def get_data(begin_time, end_time, client_usdf, client_idf=None):
    table_dict = {
        "m1m3_hp_actuator": "lsst.sal.MTM1M3.hardpointActuatorData",
        # "m1m3_hp_monitor": "lsst.sal.MTM1M3.hardpointMonitorData",
        # "mtmount_elevation": "lsst.sal.MTMount.elevation",
        "m1m3_accel": "lsst.sal.MTM1M3.accelerometerData",
        "m1m3_ims": "lsst.sal.MTM1M3.imsData"
    }
    cols_dict = {
        "m1m3_hp_actuator": ["*"],
        "m1m3_accel": ["*"],
        "m1m3_ims": ["*"]
    }
    query_dict = {}
    for key in table_dict.keys():
        query_dict[key] = []
        if (end_time < datetime(2023, 11, 28)) & (client_idf is not None):
            client = client_idf
        else:
            client = client_usdf
            
        query = getEfdData(
                    client=client,
                    topic=table_dict[key],
                    columns=cols_dict[key],
                    begin=begin_time,
                    end=end_time,
                    prePadding=0,
                    postPadding=0,
                )
        query = add_timestamp(query)

        query_dict[key].append(query)
        
    for key in table_dict.keys():
        query_dict[key] = pd.concat(query_dict[key])
    # sorted_keys = sorted(query_dict.keys())
    # initial_key = sorted_keys[0]
    # merged_df = query_dict[initial_key].sort_index()
    # # Define your tolerance for matching
    # tolerance = 0.03  # pd.Timedelta('0.03 seconds')

    # # Iterate over the remaining DataFrames and merge them
    # for key in sorted_keys[1:]:
    #     merged_df = pd.merge_asof(
    #         merged_df,
    #         query_dict[key].sort_index(),
    #         left_on="snd_timestamp_utc",
    #         right_on="snd_timestamp_utc",
    #         tolerance=tolerance,
    #         direction="nearest",
    #         suffixes=("", "_" + key),
    #     )
    # merged_df.reset_index(drop=True, inplace=True)

    return query_dict

In [ ]:
client_usdf = EfdClient("usdf_efd")
client_idf = EfdClient("idf_efd")

In [ ]:
begin_time_on=Time('2023-12-07 14:33:00.00')
end_time_on=Time('2023-12-07 14:50:00.00')

begin_time_off=Time('2023-12-07 14:20:00.00')
end_time_off=Time('2023-12-07 14:30:00.00')

In [ ]:
query_dict={}
query_dict["on"]=get_data(begin_time_on, end_time_on, client_usdf)
query_dict["off"]=get_data(begin_time_off, end_time_off, client_usdf)

In [ ]:
fig, axs = plt.subplots(3,1, sharex=True, figsize=(12,5), dpi=175)


for key in ["on","off"]:
    data_df=query_dict[key]["m1m3_ims"]
    for ax_id, val in enumerate(["x","y","z"]):
        ax=axs[ax_id]
        ax.plot(Time(data_df["snd_timestamp_utc"], format="unix").datetime, 
                data_df[f"{val}Position"] * 1e6,
                label=f"FCUs {key}")
        meanv = np.mean(data_df[f"{val}Position"] * 1e6)
        ax.set_ylabel(f"{val}Position [$\mu$m]")
        if val == "z":
            ax.set(ylim=(meanv - 4 ,meanv + 20), yticks=np.arange(int(meanv-4), meanv + 20, 5))
        else:
            ax.set(ylim=(meanv - 4 ,meanv + 4))
        if ax_id == 0:
            ax.legend()
axs[0].set_title("IMS postions")
axs[2].set_xlabel("Time [s]")

    # ax=axs[1]
    # for val in ["x","y","z"]:
    #     ax.plot(data_df["snd_timestamp_utc"], np.degrees(data_df[f"{val}Rotation"]))
# for i in range(3):
#     ax=axs[i]
#     ax.plot(vms_times_on, vms_m1m3_data_on[f"total_{i+1}"], label=f"FCUs on")
#     ax.plot(vms_times_off, vms_m1m3_data_off[f"total_{i+1}"], label=f"FCUs off")
    
#     ax.set_yticks(np.arange(0.000,0.005,0.001))
#     ax.set_ylim(-0.001,0.005)
# plt.subplots_adjust(hspace=0)
# axs[0].legend()
# axs[1].set_ylabel("VMS total acceleration [m/s^2] or [mg/s^2]?\nChannel 2")
# axs[0].set_ylabel("Channel 1")
# axs[2].set_ylabel("Channel 3")
# ax.set_xlabel("Time")
# fig.suptitle(f"M1M3 FCU #1\n{begin_time.iso[:10]} {begin_time_off.iso[11:19]}-{end_time_on.iso[11:19]}\n Total Vibration", y=0.99)


In [ ]:
fig, axs = plt.subplots(3,1, sharex=True, figsize=(12,5), dpi=175)


for key in ["on","off"]:
    data_df=query_dict[key]["m1m3_ims"]
    for ax_id, val in enumerate(["x","y","z"]):
        ax=axs[ax_id]
        ax.plot(Time(data_df["snd_timestamp_utc"], format="unix").datetime, 
                np.degrees(data_df[f"{val}Rotation"]),
                label=f"FCUs {key}")
        meanv = np.degrees(np.mean(data_df[f"{val}Rotation"] ))
        ax.set_ylabel(f"{val}Rotation [deg]")
        minv,maxv=ax.get_ylim()
        ax.set_yticks(np.arange(np.round(minv,4),maxv, 0.0005))
        # if val == "z":
        #     ax.set(ylim=(meanv - 4 ,meanv + 20), yticks=np.arange(int(meanv-4), meanv + 20, 5))
        # else:
        #     ax.set(ylim=(meanv - 4 ,meanv + 4))
        # ax.set(ylim=(meanv ,meanv + 4))
        if ax_id == 0:
            ax.legend()
axs[0].set_title("IMS Rotations")
axs[2].set_xlabel("Time [s]")

In [ ]:
"m1m3_accel": "lsst.sal.MTM1M3.accelerometerData",

In [ ]:
fig, axs = plt.subplots(6,1, sharex=True, figsize=(12,10), dpi=175)


for key in ["on","off"]:
    data_df=query_dict[key]["m1m3_hp_actuator"]
    for ax_id, val in enumerate([f"measuredForce{i}" for i in range(6)]):
        ax=axs[ax_id]
        ax.plot(Time(data_df["snd_timestamp_utc"], format="unix").datetime, 
                data_df[val], label=f"FCUs {key}")
        ax.set_ylabel(f"HP_{ax_id}\nForce [N]")
        minv,maxv=ax.get_ylim()
        ax.set_yticks(np.arange(np.round(minv/100,0)*100,np.round(maxv/100,0)*100 + 50, 50))
        #ax.set_yticks(np.arange(np.round(minv,4),maxv, 0.0005))
        # if val == "z":
        #     ax.set(ylim=(meanv - 4 ,meanv + 20), yticks=np.arange(int(meanv-4), meanv + 20, 5))
        # else:
        #     ax.set(ylim=(meanv - 4 ,meanv + 4))
        # ax.set(ylim=(meanv ,meanv + 4))
        if ax_id == 0:
            ax.legend()

axs[0].set_title("Hardpoint Forces")
axs[5].set_xlabel("Time [s]")
plt.subplots_adjust(hspace=0)

In [ ]:
fig, axs = plt.subplots(6,1, sharex=True, figsize=(12,10), dpi=175)


for key in ["on","off"]:
    data_df=query_dict[key]["m1m3_hp_actuator"]
    for ax_id, val in enumerate([f"measuredForce{i}" for i in range(6)]):
        ax=axs[ax_id]
        freq,accel_psd=get_freq_psd(data_df[val], np.mean(np.diff(data_df["snd_timestamp_utc"])))
        sel = (freq > 1)
        freq = freq[sel]
        accel_psd = accel_psd[sel]
        
        accel_psd = np.sqrt(accel_psd * np.mean(np.diff(freq)))
        ax.plot(freq, accel_psd, 
                label=f"FCUs {key}")
        ax.set_ylabel(f"HP_{ax_id}\nForce [N/Hz]")
        minv,maxv=ax.get_ylim()
        # ax.set_yticks(np.arange(np.round(minv/100,0)*100,np.round(maxv/100,0)*100 + 50, 50))
        #ax.set_yticks(np.arange(np.round(minv,4),maxv, 0.0005))
        # if val == "z":
        #     ax.set(ylim=(meanv - 4 ,meanv + 20), yticks=np.arange(int(meanv-4), meanv + 20, 5))
        # else:
        #     ax.set(ylim=(meanv - 4 ,meanv + 4))
        # ax.set(ylim=(meanv ,meanv + 4))
        if ax_id == 0:
            ax.legend()

axs[0].set_title("Hardpoint Forces")
axs[5].set_xlabel("Hz")
plt.subplots_adjust(hspace=0)

In [ ]:
query_dict[key]["m1m3_accel"].columns

In [ ]:

axis_table = { 0: "1_x",
               1: "1_y",
               2: "2_x",
               3: "2_y",
               4: "3_x",
               5: "3_y", 
               6: "4_x",
               7: "4_y",
}
fig, axs = plt.subplots(8,1, sharex=True, figsize=(12,10/6*8), dpi=175)
for key in ["on","off"]:
    data_df=query_dict[key]["m1m3_accel"]
    for ax_id, val in enumerate([f'accelerometer{i}' for i in range(8)]):
        ax=axs[ax_id]
        ax.plot(Time(data_df["snd_timestamp_utc"], format="unix").datetime, 
                data_df[val], label=f"FCUs {key}")
        ax.set_ylabel(f"accel {axis_table[ax_id]}\n[m/s^2]")
        minv,maxv=ax.get_ylim()
        #ax.set_yticks(np.arange(np.round(minv/100,0)*100,np.round(maxv/100,0)*100 + 50, 50))
        ax.set_yticks(np.linspace(np.round(minv,3),np.round(maxv,3), 4))
        # if val == "z":
        #     ax.set(ylim=(meanv - 4 ,meanv + 20), yticks=np.arange(int(meanv-4), meanv + 20, 5))
        # else:
        #     ax.set(ylim=(meanv - 4 ,meanv + 4))
        # ax.set(ylim=(meanv ,meanv + 4))
        if ax_id == 0:
            ax.legend()
axs[0].set_title("DC acceleromteters")
axs[7].set_xlabel("Time [s]")
plt.subplots_adjust()
plt.savefig("./plots/dc_accel_telemetry.png")

In [ ]:

axis_table = { 0: "1_x",
               1: "1_y",
               2: "2_x",
               3: "2_y",
               4: "3_x",
               5: "3_y", 
               6: "4_x",
               7: "4_y",
}
fig, axs = plt.subplots(8,1, sharex=True, figsize=(12,10/6*8), dpi=175)
for key in ["on","off"]:
    data_df=query_dict[key]["m1m3_accel"]
    for ax_id, val in enumerate([f'accelerometer{i}' for i in range(8)]):
        ax=axs[ax_id]
        freq,accel_psd=get_freq_psd(data_df[val], np.mean(np.diff(data_df["snd_timestamp_utc"])))
        sel = (freq > 1)
        freq = freq[sel]
        accel_psd = accel_psd[sel]
        
        accel_psd = np.sqrt(accel_psd * np.mean(np.diff(freq)))
        ax.plot(freq, accel_psd, 
                label=f"FCUs {key}")
        ax.set_ylabel(f"accel {axis_table[ax_id]}\n[(m/s^2)/Hz]")
        # ax.set_ylabel(f"accel {axis_table[ax_id]}\n[m/s^2]")
        # minv,maxv=ax.get_ylim()
        #ax.set_yticks(np.arange(np.round(minv/100,0)*100,np.round(maxv/100,0)*100 + 50, 50))
        # ax.set_yticks(np.linspace(np.round(minv,3),np.round(maxv,3), 4))
        # if val == "z":
        #     ax.set(ylim=(meanv - 4 ,meanv + 20), yticks=np.arange(int(meanv-4), meanv + 20, 5))
        # else:
        #     ax.set(ylim=(meanv - 4 ,meanv + 4))
        # ax.set(ylim=(meanv ,meanv + 4))
        if ax_id == 0:
            ax.legend()
axs[0].set_title("DC acceleromteters")
axs[7].set_xlabel("Time [s]")
plt.subplots_adjust(hspace=0)
plt.savefig("./plots/dc_accel_psd.png")